In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

template = PromptTemplate.from_template("{country_a}와 {country_b} 사이의 거리는 몇 km인가요? 당신 역할도 알려주세요.")
prompt = template.format(country_a="멕시코", country_b="태국")


AIMessage(content='멕시코와 태국 사이의 거리는 대략 16,000km입니다. 저는 여행 관련 정보를 제공하는 인공지능 어시스턴트입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 44, 'total_tokens': 97, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CaHwZa0QITs2LDqTCf6gDvrSHeb7Y', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--985c41ee-6d48-46c7-86c9-d19fadf0a629-0', usage_metadata={'input_tokens': 44, 'output_tokens': 53, 'total_tokens': 97, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [6]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert. And you only reply in {language}."),
    ("ai", "저는 {expert_class}입니다."),
    ("human", "{country_a}와 {country_b} 사이의 거리는 몇 km인가요? 당신의 역할도 알려주세요.")
])

prompt = template.format_messages(
    language="Korean",
    expert_class="지리 전문가",
    country_a="멕시코",
    country_b="태국"
)

chat.invoke(prompt)

AIMessage(content='멕시코와 태국 사이의 거리는 약 16,000km입니다. 제 역할은 지리 전문가입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 76, 'total_tokens': 114, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CaHx7ODHEcfdIoMFCazIKkGp2Vxt9', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--3e08fd78-3c5a-454a-af00-789d5bb50843-0', usage_metadata={'input_tokens': 76, 'output_tokens': 38, 'total_tokens': 114, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [7]:
from langchain_core.output_parsers import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        return [s for i in text.split(",") if (s := i.strip())]
    
parser = CommaOutputParser()
parser.parse("hello, how, are,    ")

chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients."),
    ("human", "저는 {cuisine}을 요리하고 싶어요.")
])
chef_chain = chef_prompt | chat

veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."),
    ("human", "{recipe}")
])
veg_chef_chain = veg_chef_prompt | chat

final_chain = {"recipe": chef_chain} | veg_chef_chain
final_chain.invoke({
    "cuisine": "칼국수"
})



AIMessage(content='한국의 전통 음식인 칼국수 레시피를 살펴보니 육수에 사용되는 소고기 뼈가 대체할 수 있는 재료가 필요합니다. 소고기 뼈를 대체할 수 있는 옵션은 식물성 재료로 만든 육수 또는 버섯 육수입니다. 소고기 뼈의 풍부한 맛과 영양을 대체하기 위해 다음과 같이 대체재료를 사용할 수 있습니다:\n\n**소고기 뼈 대체재료:**\n\n1. **버섯 육수:** 건조된 버섯(표고버섯, 새송이버섯 등)을 물에 불려 육수를 내는 방법이 있습니다. 버섯은 풍부한 맛과 향을 제공하여 칼국수의 국물을 풍부하고 깊은 맛으로 만들어줍니다. 물에 버섯을 불려 육수를 만들어 사용하면 소고기 뼈의 부재를 대체할 수 있습니다.\n\n버섯 육수를 만들기 위해서는 건조된 버섯을 물에 충분히 불려 부드럽게 만든 후, 끓는 물에 버섯과 함께 넣어 육수를 내면 됩니다. 버섯의 풍부한 맛이 칼국수에 훌륭한 대체재료가 될 것입니다.\n\n이 외에 소고기 뼈를 대체할 수 있는 다른 방법이 필요하다면 추가로 문의해주세요.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 450, 'prompt_tokens': 901, 'total_tokens': 1351, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CaID6W288ruVJT44uM4SDOeH1tmoC